In [1]:
import pymysql
import pandas as pd
import re

people_db = pymysql.connect(
    user='1zo',
    passwd='multi1234!',
    host='13.112.232.65',
    db='tweets',
    charset='utf8mb4'
)

cursor = people_db.cursor(pymysql.cursors.DictCursor)

sql = "SELECT DISTINCT * FROM tweet_person"
cursor.execute(sql)
people_db = cursor.fetchall()
people_db

[{'name': '메인트 트친소 김허브🌱',
  'text': '#본사람다하기\n\n본명 : 알려주겠냐\n나이 : 만 14세\n키 : 18n\n몸무게 : 7n\nMBTI :I/ENFP\n지갑 잔고 : 2만 5천원\n통장 잔고 : 모름\n거주지 : 외국 \n폰기종: 갤럭시 S21+\n취미: 노래 듣기, 와일드리프트, 사진 찍기, 화분 키우기',
  'created_at': '2022-02-18T02:47:36.000Z',
  'author_id': '1482353272149405697',
  'username': '08H_erb',
  'id': '1494503840322064384'},
 {'name': '류예은',
  'text': '#본사람다하기 \n본명 : 안 알려줌\n나이 : 안 알려줌\n키 : 167.8\n몸무게 : 안 알려줌\nMBTI : ISTJ\n지갑 잔고 : 지갑 안 씀\n통장 잔고 : 25만원\n거주지 : 경기도\n폰기종: S20\n취미: 프리채널, 잠자기',
  'created_at': '2022-02-18T02:36:32.000Z',
  'author_id': '1479775692716150786',
  'username': '0harin',
  'id': '1494501054922489856'},
 {'name': 'ミンソのツイッター',
  'text': '#본사람다하기 \n본명: 닉넴과 동일\n나이: 19\n키: 161\n몸무게: 46\nMBTI: ESFJ\n지갑 잔고 : ないしょ\n통장 잔고 : ?\n거주지: 경기도 어딘가?\n폰기종: 아이폰7\n취미: 애니보기, 앙스타 게임하기, 놀러다니기',
  'created_at': '2022-02-18T04:22:24.000Z',
  'author_id': '1239434750277771264',
  'username': 'mintang2728',
  'id': '1494527697267347458'},
 {'name': 'Espacer (에스파시/エスパシ)',
  'text': '#본사람

In [3]:
len(people_db)

1029

In [2]:
print(people_db[0]['text'])

#본사람다하기

본명 : 알려주겠냐
나이 : 만 14세
키 : 18n
몸무게 : 7n
MBTI :I/ENFP
지갑 잔고 : 2만 5천원
통장 잔고 : 모름
거주지 : 외국 
폰기종: 갤럭시 S21+
취미: 노래 듣기, 와일드리프트, 사진 찍기, 화분 키우기


In [4]:
#기본 데이터프레임 만들기
def BasicDF(colname):
    col = ["id",colname]
    df = pd.DataFrame(columns = col)
    return df

df_mbti = BasicDF('mbti')
df_age = BasicDF('age')
df_height = BasicDF('height')

In [5]:
#mbti 전처리
mbti_list = ['ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 
              'ESTP', 'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP']
col = [df_mbti.columns[0],df_mbti.columns[1]]
for mbti in people_db:
    temp = {}
    mbti['username'] = mbti['username'].replace(' ','')
    for i in range(len(col)):
        if i == 0:
            temp[col[i]] = mbti['username']
        elif i != 0:
            text = mbti['text']
            m = re.search('MBTI', text)
            if m :
                text = text[m.span()[1]+2:m.span()[1]+7]
                text = text.replace(" ", "")
                text = text.replace("\n", "")
                text = text.upper()
                if text in mbti_list:
                    temp[col[i]] = text
    df_mbti = df_mbti.append(temp, ignore_index=True)

In [6]:
#age 전처리
col = [df_age.columns[0],df_age.columns[1]]
for age in people_db:
    temp = {}
    age['username'] = age['username'].replace(" ",'')
    for i in range(len(col)):
        if i == 0:
            temp[col[i]] = age['username']
        elif i != 0:
            text = age['text']
            data = re.search('나이', text)
            if data :
                text = text[data.span()[1]+2:data.span()[1]+7]
                text = text.replace(" ", "")
                text = text.replace("\n", "")
                if '년생' in text:
                    n = re.findall(r'\d+', text)
                    if int(n[0]) > 22:
                        text = str(int(2022)-int('19'+n[0])+1)
                    else: 
                        text = str(int(2022)-int('20'+n[0])+1)
                if text[0] == '0':
                    n = re.findall(r'\d+', text)
                    text = str(int(2022)-int('20'+n[0])+1)
                numbers = re.findall(r'\d+', text)
                if len(numbers) > 0 : 
                    if 8 < int(numbers[0]) and int(numbers[0]) < 80:
                        temp[col[i]]=numbers[0]
                    else : pass
    df_age = df_age.append(temp, ignore_index=True)

In [7]:
#height 전처리
col = [df_height.columns[0],df_height.columns[1]]
for height in people_db:
    temp = {}
    height['username'] = height['username'].replace(' ','')
    for i in range(len(col)):
        if i == 0:
            temp[col[i]] = height['username']
        elif i != 0:
            text = height['text']
            data = re.search('키', text)
            if data :
                text = text[data.span()[1]+2:data.span()[1]+7]
                text = text.replace(" ", "")
                text = text.replace("\n", "")
                numbers = re.findall(r'\d+', text)
                if len(numbers) > 0 : 
                    if int(numbers[0]) > 100:
                        temp[col[i]] = numbers[0]
    df_height = df_height.append(temp, ignore_index=True)    

In [8]:
dfdf = pd.merge(df_mbti,df_age)
total_new = pd.merge(dfdf,df_height)
total_new = total_new.drop_duplicates(['id'])
display(total_new)

,id,mbti,age,height
0,08H_erb,NaN,14,NaN
8,0harin,ISTJ,NaN,167
9,mintang2728,ESFJ,19,161
10,Stella91114207,ISTP,16,NaN
11,zzangboy98,ISFJ,25,181
...,...,...,...,...
1288,an__onnn,ISFP,29,158
1289,mateu0830_0314,ISFP,18,163
1290,muddthejn,NaN,NaN,160
1291,HeheKoean,ISTP,NaN,171


In [24]:
total_new.to_csv("./output/mbti_age_height.csv")

In [9]:
#DB에 저장
from sqlalchemy import create_engine

db_connection_str = 'mysql+pymysql://'+'1zo'+':'+'multi1234!'+'@'+'13.112.232.65'+'/'+'tweets'
# 'mysql+pymysql://[db유저이름]:[db password]@[host address]/[db name]'
db_connection = create_engine(db_connection_str)
conn = db_connection.connect()
total_new.to_sql(name='people_db', con=db_connection, if_exists='replace',index=False)  

In [10]:
#입맛궁합 가져오기
food = pd.read_csv("./output/id_food.csv")
food.drop(["Unnamed: 0","떡볶이",], axis=1, inplace=True)
food

,id,육회,마라탕,간장게장,선지,민초,탄산수,커피,생굴,닭발,막창,곱창,닭똥집,알탕,산낙지,오뎅,양꼬치
0,Adorable_Lana,3,2,1.0,2.0,3.0,3.0,5.0,1.0,3.0,5.0,5.0,4.0,2.0,4.0,4.0,5.0
1,kumahimeinlove,3,NaN,2.0,3.0,5.0,4.0,3.0,2.0,3.0,3.0,2.0,3.0,3.0,3.0,4.0,NaN
2,NangmanLaws,5,4,4.0,2.0,2.0,3.0,2.0,3.0,2.0,4.0,4.0,4.0,3.0,3.0,3.0,5.0
3,iamnotlogical_,NaN,3,5.0,2.0,4.0,1.0,3.0,5.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0,3.0
4,OuO76151129,NaN,3,2.0,0.0,0.0,3.0,0.0,3.0,4.0,5.0,1.0,2.0,0.0,3.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4743,juyonisaranghae,1,2,0.0,0.0,5.0,NaN,5.0,0.0,2.0,1.0,0.0,2.0,0.0,NaN,3.0,1.0
4744,KIM_Ryan10,2,3,4.0,2.0,3.0,NaN,0.0,1.0,4.0,3.0,3.0,4.0,4.0,4.0,2.0,4.0
4745,ssuing34,3,3,0.0,0.0,4.0,NaN,3.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0
4746,CELE5TIA,4,3,1.0,0.0,4.0,3.0,5.0,0.0,1.0,5.0,5.0,5.0,0.0,3.0,4.0,5.0


In [11]:
#본사람다하기+입맛궁합
people_new = pd.merge(total_new, food)
people = people_new.drop_duplicates(['id'])
# people_new.to_csv("./output/total.csv")
display(people)

,id,mbti,age,height,육회,마라탕,간장게장,선지,민초,탄산수,커피,생굴,닭발,막창,곱창,닭똥집,알탕,산낙지,오뎅,양꼬치
0,16_5ii,ENFP,17,165,5,5,1.0,0.0,2.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1,cloudteemo,ENFJ,20,NaN,3,3,2.0,0.0,3.0,2.0,4.0,0.0,3.0,3.0,3.0,3.0,3.0,2.0,NaN,2.0
2,sunkist_0412,INFP,NaN,162,1,NaN,0.0,3.0,3.0,4.0,0.0,5.0,4.0,4.0,3.0,0.0,3.0,2.0,0.0,NaN
3,Il_l1x,ISTP,22,155,3,3,0.0,0.0,1.0,4.0,5.0,0.0,0.0,5.0,5.0,1.0,0.0,3.0,3.0,0.0
4,Wolmi_Dal,ESFP,18,172,0,4,3.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
5,mad__iy,ENFP,NaN,161,5,NaN,0.0,5.0,3.0,0.0,0.0,5.0,5.0,5.0,4.0,0.0,2.0,3.0,3.0,NaN
6,songwhale_0317,ENTP,13,153,NaN,5,5.0,4.0,4.0,5.0,3.0,0.0,5.0,5.0,5.0,5.0,2.0,5.0,5.0,5.0
7,egn_an,INFP,NaN,NaN,3,NaN,0.0,5.0,1.0,3.0,0.0,3.0,4.0,4.0,0.0,0.0,0.0,3.0,0.0,5.0
8,Accio_Tennant,ENFP,NaN,158,5,NaN,1.0,3.0,2.0,4.0,3.0,5.0,3.0,3.0,2.0,2.0,4.0,5.0,2.0,NaN
9,loverevol_,NaN,21,173,5,NaN,2.0,1.0,1.0,4.0,NaN,4.0,4.0,4.0,NaN,4.0,5.0,3.0,NaN,4.0
